# Correct NP precipitation following Yang

The notebook describes the algorithms to correct precipitation from North Pole drifting stations following the WMO approach described in Goodison et al (1998) and Yang (1999).

In [1]:
%matplotlib inline

In [2]:
import sys
sys.path.append('../source')

import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import readers.npsnow as npsnow
from merge_npsnow_data import get_station_list, met_filename, get_precip, plot_station_met, merge_one_station

## Data
I use raw data from the NP drifting stations.  I have produced my own combined files that combine precipitation and meteorological observations.  These files are stored in /home/apbarret/Data/NPSNOW/my_combined_met

- TAIR - 2m air temperature
- RH - Relative humidity at 2m
- SLP - Sea level pressure
- WDIR - wind direction
- WSPD - wind speed m/s
- TOTCLD - total cloud cover
- LOWCLD - low cloud cover
- TSURF - surface temperature
- PRECIP - precipitation amount in mm  0 = Trace, when PTYPE != 0
- PTYPE - precipitation type
  - 1 - solid precipitation
  - 2 - mixed phase (rain/snow)
  - 3 - rain
- SDEPTH - snow depth

In [3]:
data_dir = '/home/apbarret/Data/NPSNOW/my_combined_met'
filelist = glob.glob(os.path.join(data_dir, 'npmet_??_combined.csv'))

## Merge met and precip data
The met and precip files are resampled and merged into a single dataframe.  Daily maximum and minimum air temperatures are also added to the structure.  

In [15]:
sid = '21'
met = npsnow.read_met(met_filename(sid))
precDay = get_precip(sid) 
metDay = met.resample('D').mean()
metDay['TMIN'] = met['TAIR'].resample('D').min()
metDay['TMAX'] = met['TAIR'].resample('D').max()
merged_df = pd.concat([metDay,precDay], axis=1, sort=False)
merged_df = merged_df.rename({'amount': 'PRECIP', 'type': 'PTYPE'}, axis=1)
df = merged_df.drop('statid', axis=1)
df['Ug'] = df.apply(wind_at_gauge, axis=1)
df.head()

,Station_ID,Latitude,Longitude,TAIR,RH,SLP,WDIR,WSPD,TOTCLD,LOWCLD,TSURF,TMIN,TMAX,PRECIP,PTYPE
1972-05-01,21.0,NaN,NaN,-15.671429,90.000,1018.614286,288.571429,2.428571,2.500,0.000,NaN,-19.0,-13.3,0.0,1.0
1972-05-02,21.0,NaN,NaN,-15.412500,90.000,1022.000000,260.000000,2.375000,5.000,0.000,NaN,-19.5,-11.1,NaN,NaN
1972-05-03,21.0,NaN,NaN,-14.400000,87.500,1027.337500,115.000000,1.375000,0.000,0.000,NaN,-17.8,-11.3,NaN,NaN
1972-05-04,21.0,NaN,NaN,-14.837500,89.500,1026.950000,67.500000,4.000000,0.625,0.000,NaN,-18.5,-12.2,NaN,NaN
1972-05-05,21.0,NaN,NaN,-13.912500,88.875,1021.050000,295.000000,4.750000,6.375,5.375,NaN,-19.1,-12.5,0.9,1.0


## Generate table to match Yang monthly tables - wind correction, trace correction and corrected precip left as Nan

In [45]:
dfMon = pd.DataFrame({
    'ND': df.Station_ID.resample('MS').count(),
    'Tmn': df.TAIR.resample('MS', label='left').mean(),
    'Ug': df.WSPD[df.WSPD < 6.].resample('MS', label='left').mean(),
    'DP': df.PRECIP[df.PRECIP > 0.].resample('MS').count(),
    'Dtc': df.PRECIP[df.PRECIP == 0.].resample('MS').count(),
    'Pg': df.PRECIP.resample('MS').sum()
})
dfMon

,ND,Tmn,Ug,DP,Dtc,Pg
1972-05-01,31,-8.381336,3.683929,7,13,12.9
1972-06-01,30,-3.084167,3.868056,20,7,19.4
1972-07-01,31,0.010887,4.125000,19,7,30.0
1972-08-01,31,-0.777823,4.161458,28,2,60.6
1972-09-01,30,-6.714583,2.685000,24,1,22.8
1972-10-01,31,-15.946371,3.363636,30,0,24.3
1972-11-01,30,-27.625417,2.615000,15,2,9.6
1972-12-01,31,-27.618145,3.351190,20,1,21.0
1973-01-01,31,-34.188710,2.754310,17,7,11.3
1973-02-01,28,-34.441071,3.206731,10,5,6.8


f = filelist[0]
df = pd.read_csv(f, header=0, index_col=0, sep=',', parse_dates=True)
df.head()

In [ ]:
plot_station_met(df, title=os.path.basename(f))

In [17]:
def cr_tretyakov_snow(ws, tmax, tmin):
    """Calculates the catch ratio of a Tretyakov rain gauge using WMO standard procedure for snow
    
    Methods for developing catch ratios are described in Goodison et al 1998.
    
    Arguments
    ---------
    ws - wind speed at height of gauge orifice in m/s
    tmax - maximum air temperature in degrees celsius
    tmin - minimum air temperature
    """
    return 103.11 - 8.67*ws + 0.3*tmax


def cr_tretyakov_mixed(ws, tmax, tmin):
    """Calculates the catch ratio of a Tretyakov rain gauge using WMO standard procedure for mixed precipitation
    
    Methods for developing catch ratios are described in Goodison et al 1998.
    
    Arguments
    ---------
    ws - wind speed at height of gauge orifice in m/s
    tmax - maximum air temperature in degrees celsius
    tmin - minimum air temperature in degrees celsius
    """
    return 96.99 - 4.46*ws + 0.88*tmax +0.22*tmin

def cr_tretyakov_rain(ws, tmax, tmin):
    """Returns catch ratio for Treyakov rain gauge using WMO standard procedure for rain.
    
    In Goodison et al 1998, no rain correction is available but they state that catch ratios for rain are 
    largely unaffected by wind.  So I take the average catch ratio of all sites from Table 4.4.1
    
    I drop catch ratios from Bismark and Harzgerode because these a lower by more than 5% than other locations."""
    return 91.7


def cr_tretyakov_dry(ws, tmax, tmin):
    return 100.

def catch_ratio(x):
    cr_func = {
        0: cr_tretyakov_dry,
        1: cr_tretyakov_snow,
        2: cr_tretyakov_mixed,
        3: cr_tretyakov_rain,
        }
    if x.WSPD < 6.:
        return 1./ (cr_func[x.PTYPE](x.WSPD, x.TMAX, x.TMIN)*0.01 )
    else:
        return 1.


def wind_at_gauge(x):
    """Reduces 10 m wind speed to wind at gauge height orifice"""
    H = 10.  # height of anenometer
    hg = 3.  # height of gauge orifice
    z0 = 0.01  # Roughness parameter of snow surface
    return x.WSPD * np.log10(hg/z0) / np.log10(H/z0)

Proof of concept for using average catch ratio for rain

In [ ]:
rain = pd.DataFrame({'wspd': [3.8, 2.7, 1.0, 2.5, 4.2, 3.3, 1.2, 1.6, 1.9, 3.9, 2.3],
                     'cr': [91.4, 92.0, 94.3, 86.6, 81.3, 71.6, 90.6, 88.2, 95.0, 97.4, 90.0]})
rain.drop([4,5]).mean()

In [ ]:
rain.plot(x='wspd', y='cr', marker='+', ls='')

Set Trace days to 0.1 mm

In [25]:
df['PTRACE'] = 0.0
df.loc[(df['PTYPE'] > 0) & (df['PRECIP'] == 0.), 'PTRACE'] = 0.1
df['Ug'] = df.apply(wind_at_gauge, axis=1)
df['Ug'].where(df['Ug'] <= 6.).resample('M').mean()

1972-05-31    3.331677
1972-06-30    3.914169
1972-07-31    3.695039
1972-08-31    3.715682
1972-09-30    2.465653
1972-10-31    3.272247
1972-11-30    2.517669
1972-12-31    3.145944
1973-01-31    2.457144
1973-02-28    2.647820
1973-03-31    3.273339
1973-04-30    2.549371
1973-05-31    3.077971
1973-06-30    3.307520
1973-07-31    3.119338
1973-08-31    3.729000
1973-09-30    3.192243
1973-10-31    3.018102
1973-11-30    3.264601
1973-12-31    3.332318
1974-01-31    3.697732
1974-02-28    2.862187
1974-03-31    3.455584
1974-04-30    3.082165
Freq: M, Name: Ug, dtype: float64

In [ ]:
df['CR'] = df.apply(catch_ratio, axis=1)
df['PWIND'] = df['PRECIP'] * df['CR']
df.head()

In [ ]:
df['PCORR'] = df.PRECIP + df.PTRACE + df.PWIND
df.head()

In [ ]:
precMon = df[['PRECIP', 'PTRACE', 'PWIND', 'PCORR']].resample('M').sum()
precMon['TAIR'] = df['TAIR'].resample('M').mean()
precMon['WSPD'] = df['WSPD'].resample('M').mean()
precMon['TAIR_yang'] = yangMon['Tmn'].values
precMon['WSPD_yang'] = yangMon['Ug'].values
precMon

In [ ]:
sub = df[(df.index.year == 1972) & (df.index.month == 5)]
(sub.PTRACE * sub.CR).sum()

In [42]:
yang_diri = '/home/apbarret/Data/NPSNOW/yang_precip'
yangMon = npsnow.read_yang_updated(os.path.join(yang_diri, f'yang_np_precip_updated_coords_{sid}.csv'))
yangMon.index = yangMon.Date
yangMon

,Date,NP,ND,Tmn,Ug,DP,Dtc,snow%,Pg,windC,traceC,Pc,Lat,Lon
Date,,,,,,,,,,,,,,
1972-05-01,1972-05-01,21.0,30.0,-8.1,3.8,7.0,22.0,100.0,12.9,8.5,2.2,23.6,74.893750,175.057396
1972-06-01,1972-06-01,21.0,30.0,-3.1,3.8,19.0,10.0,100.0,17.6,6.7,1.0,25.3,75.552982,170.826754
1972-07-01,1972-07-01,21.0,31.0,0.0,3.3,19.0,11.0,30.3,30.0,4.2,1.1,35.3,75.984938,171.120617
1972-08-01,1972-08-01,21.0,31.0,-0.7,3.5,27.0,3.0,81.9,60.3,16.9,0.3,77.5,77.036087,169.564638
1972-09-01,1972-09-01,21.0,30.0,-6.4,2.5,23.0,6.0,97.3,21.9,7.6,0.6,30.1,77.683627,164.462059
1972-10-01,1972-10-01,21.0,31.0,-15.2,3.3,29.0,1.0,100.0,23.5,11.1,0.1,34.7,77.927540,167.542778
1972-11-01,1972-11-01,21.0,30.0,-26.9,2.3,14.0,15.0,100.0,9.1,3.5,1.5,14.1,77.696207,166.222874
1972-12-01,1972-12-01,21.0,31.0,-26.4,3.5,20.0,10.0,100.0,21.0,18.1,1.0,40.1,77.825287,161.990057
1973-01-01,1973-01-01,21.0,31.0,-32.7,2.2,17.0,13.0,100.0,11.3,3.1,1.3,15.7,78.390517,161.705862
